 # Data Challenge Project 2019

                                                                                                                                    Γιώργος Λεβέντης(3130114)

## Επεξεργασία dataset

Αρχικά,ξεκινάμε κάνοντας import τα κατάλληλα πακέτα για την εισαγωγή και επεξεργασία των δοσμένων αρχείων train.csv και test.csv και τα φορτώνουμε σε Data Frames

In [1]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('../project/train.csv')
df_test = pd.read_csv('../project/test.csv')

df_train.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,PAX
0,2011-12-05,ORD,Chicago,41.978603,-87.904842,EWR,Newark,40.692500,-74.168667,8.352941,5.667243,7
1,2013-03-01,ATL,Atlanta,33.636719,-84.428067,LGA,New York,40.777245,-73.872608,10.421053,10.001754,7
2,2012-02-06,ORD,Chicago,41.978603,-87.904842,BOS,Boston,42.364347,-71.005181,9.250000,7.136821,7
3,2012-12-10,ORD,Chicago,41.978603,-87.904842,SFO,San Francisco,37.618972,-122.374889,8.666667,7.404291,7
4,2012-04-13,SFO,San Francisco,37.618972,-122.374889,JFK,New York,40.639751,-73.778925,14.037037,9.858544,7


In [2]:
df_train.shape

(8899, 12)

Χωρίζουμε το training set στις πληροφοριές και στην μεταβλήτη που θέλουμε ο ταξινομητής μας να προβλέπει.

In [3]:
Y = df_train[['PAX']]

Y.head()

,PAX
0,7
1,7
2,7
3,7
4,7


Από τις πληροφορίες που μας δίνονται δημιουργούμε μερικές νέες και διαγράφουμε κάποιες που δεν θα βοηθήσουν τόσο πολύ τον ταξινομητή.
Χρησιμοποιώντας την DateOfDeparture και την βιβλιοθήκη datetime μπορούμε να 'σπάσουμε' την ημερομηνία σε εποχή,χρόνο,μήνα,μέρα και μέρα της εβδομάδας(1 για Δευτέρα,2 για Τρίτη κλπ).
Επιπλέον χρησιμοποιώντας τις γεωγραφικές συντεταγμένες μπορούμε να υπολογίσουμε την απόσταση Haversine μεταξύ των σημειών αναχώρησης και άφιξης.
Όλα αυτά μπορούν να γίνουν γρήγορα και για τα 2 σέτ χρησιμοποιώντας μια συνάρτηση.

In [4]:
from datetime import datetime
import haversine

def process_df(df): 
    df["distance"] = df_train.apply(lambda row : haversine.haversine((row["LatitudeDeparture"], row["LongitudeDeparture"]),
        (row["LatitudeArrival"], row["LongitudeArrival"])), axis=1)

    year = lambda x : datetime.strptime(x, '%Y-%m-%d').year
    df['year'] = df['DateOfDeparture'].map(year)

    month = lambda x : datetime.strptime(x , '%Y-%m-%d' ).month
    df['month'] = df['DateOfDeparture'].map(month)

    day = lambda x :  datetime.strptime(x , '%Y-%m-%d' ).day
    df['day'] = df['DateOfDeparture'].map(day)

    weekdays = lambda x : datetime.strptime(x , '%Y-%m-%d' ).isoweekday()
    df['weekday'] = df['DateOfDeparture'].map(weekdays)
    
    season = {11: 'Winter', 12: 'Winter', 1: 'Winter', 2: 'Spring', 3: 'Spring', 4: 'Spring', 
              5: 'Summer', 6: 'Summer', 7: 'Summer', 8: 'Autumn', 9: 'Autumn', 10: 'Autumn'}

    df['season'] = df['month'].apply(lambda x : season[x])
    
    return df

df_train = process_df(df_train)
df_test = process_df(df_test)

df_train.head()


,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,PAX,distance,year,month,day,weekday,season
0,2011-12-05,ORD,Chicago,41.978603,-87.904842,EWR,Newark,40.692500,-74.168667,8.352941,5.667243,7,1527.462493,2011,12,5,1,Winter
1,2013-03-01,ATL,Atlanta,33.636719,-84.428067,LGA,New York,40.777245,-73.872608,10.421053,10.001754,7,1180.967843,2013,3,1,5,Spring
2,2012-02-06,ORD,Chicago,41.978603,-87.904842,BOS,Boston,42.364347,-71.005181,9.250000,7.136821,7,1879.165071,2012,2,6,1,Spring
3,2012-12-10,ORD,Chicago,41.978603,-87.904842,SFO,San Francisco,37.618972,-122.374889,8.666667,7.404291,7,3832.261997,2012,12,10,1,Winter
4,2012-04-13,SFO,San Francisco,37.618972,-122.374889,JFK,New York,40.639751,-73.778925,14.037037,9.858544,7,5401.866530,2012,4,13,5,Spring


Επιλέγουμε ποιές πληροφορίες θέλουμε να χρησιμοποιήσουμε για την εκπαίδευση του ταξινομητή μας.(Στο συγκεκριμένο σημείο μετά απο αρκετό trial and error έφτασα στο συμπέρασμα οτι η πληροφορία distance δεν βοηθούσε στην εκπαίδευση και έτσι την έκανα drop)

In [5]:
df_train.shape

(8899, 18)

In [6]:
df_train.drop(df_train[['DateOfDeparture','CityDeparture','CityArrival','PAX','LongitudeDeparture','distance','std_wtd',
                        'LatitudeDeparture','LongitudeArrival','LatitudeArrival']], axis=1 , inplace = True)
    
df_test.drop(df_test[['DateOfDeparture','CityDeparture','CityArrival','LongitudeDeparture',
                     'LatitudeDeparture','LongitudeArrival','LatitudeArrival','distance','std_wtd',]], axis=1 , inplace = True)

df_train.shape

(8899, 8)

Για πιό γρήγορο έλεγχο της απόδοσης , χρησιμοποίησα το πακέτο train_test_split για να χωρίσω το dataset σε training και test set(10% του αρχικού σετ).

In [7]:
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(df_train ,Y ,  test_size = 0.1 ,random_state=0)
y_train = np.ravel(y_train)

Έχοντας κρατήσει τις πληροφόριες που θέλουμε,τώρα είναι η ώρα να τους δώσουμε την κατάλληλη μορφή για τον ταξινομητή που θα εκπαιδεύσουμε.Για τις συνεχείς τιμές θα χρησιμοποιήσουμε MinMaxScaler για τις κατηγορικές One-hot encoding.Για να γίνει οργανωμένα η μετατροπή της κάθε στήλης χρησιμοποίησα την κλάση ColumnTransformer του πακέτου scikit-learn έτσι ώστε να χωρίσω τις στήλες για την αντίστοιχη μετατροπή

In [8]:
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.compose import ColumnTransformer
    
coltransf = ColumnTransformer([('one_hot',OneHotEncoder(categories='auto',sparse=False) ,
                                ['Departure','Arrival','day','month','year','weekday','season']),
                                ('scaling', MinMaxScaler() ,['WeeksToDeparture'] )])

X_train = coltransf.fit_transform(df_train)
X_test = coltransf.transform(X_test)
df_test = coltransf.transform(df_test)

X_train.shape

(8899, 98)

In [9]:
y_train.shape

(7119,)

# Επιλογή και εκπαίδευση ταξινομητή

Μετά απο δοκιμές αρκετών ταξινομητών , κατέληξα στην κλάση MLPClassifier του πακέτου scikit-learn εφόσον ο συγκεκριμένος μου έδινε τα καλύτερα αποτελέσματα.
Οι ταξινομητές που δοκιμάστηκαν ήταν οι : KNeighborsClassifier , LogisticRegression , RandomForestClassifier και MultinomialNB.

Φορτώνουμε την κλάση MLPClassifier απο το πακέτο και δημιουργούμε ένα αντικείμενο MLPClassifier με 3 κρυφά layers απο 64,32,16 νευρώνες το καθένα.
Ύστερα εκπαιδεύουμε τον ταξινομητή μας και κάνουμε μια πρόβλεψη πάνω στα δεδομένα που πήραμε απο το train_test_split.

In [11]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(alpha=0.55 , hidden_layer_sizes=(64,32,16),random_state=0)
clf.fit(X_train, Y )
y_pred = clf.predict(X_test)

/home/geo-lev/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/geo-lev/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Φορτώνουμε την κλάση f1-score απο το πακέτο scikit-learn και βλέπουμε το score του ταξινομητή μας

In [12]:
from sklearn.metrics import f1_score
f1_score(y_pred , y_test , average='micro')

0.85

Για επιπλέον πληροφοριές σχετικά με την απόδοση του ταξινομητή μπορούμε να φορτώσουμε την κλάση cross_val_score του scikit-learn έτσι ώστε να δούμε ποσο overfitted είναι ο ταξινομητής στα δεδομένα που τον εκπαιδεύσαμε

In [14]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf , X_train , Y , cv =5 , scoring='f1_micro')

/home/geo-lev/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/geo-lev/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/geo-lev/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/geo-lev/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Max

In [16]:
scores.mean()

0.5509505166257321

In [17]:
scores.std()

0.012534503686649717

Παρ'όλο που το αρχικό f1-score ήταν 84% , το μέσο cross_val_score ήταν 55%.Η αιτία για αυτό είναι οτι ο ταξινομητής πιθανώς να έχει γίνει overfitted στα δεδομένα εκπαίδευσης.Το ποσοστό της τυπικής απόκλισης 1,2% μας δείχνει οτι ο ταξινομητής είναι αρκετά εύρωστος(robust) το οποίο είναι καλό.

# Ανέβασμα και score στην πλατφόρμα Kaggle

Για να προβλέψουμε το αρχείο που πρέπει να ανέβει στην πλατφόρμα δεν υπάρχει λόγο να χωρίσουμε το αρχικό μας dataset με την train_test_split

In [20]:
clf.fit(X_train , Y)

/home/geo-lev/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/geo-lev/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.55, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(64, 32, 16), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=0, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [24]:
y_pred = clf.predict(df_test)
y_pred.shape

(2229,)

Έπειτα χρησιμοποιούμε τον δοσμένο κώδικα απο την εκφώνηση για να δημιουργήσουμε το κατάλληλο αρχείο csv 

In [27]:
import csv 
with open('../project/y_pred.csv', 'w', newline ='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Id', 'Label'])
    for i in range(y_pred.shape[0]):
        writer.writerow([i, y_pred[i]])

In [ ]:
Ανεβάζουμε το αρχείο στην πλατφόρμα και παίρνουμε σκόρ 0.59431 στο public leaderboard και 0.58167 στο private leaderboard.